In [5]:
import io
import zipfile
import requests
import pandas as pd

WDS_BASE = "https://www150.statcan.gc.ca/t1/wds/rest"
LANG = "en"

PID_CPI_LEVELS = 18100004
PID_WEIGHTS    = 18100007

In [6]:
def get_wds_zip_url(product_id: int, lang: str = "en"):
    url = f"{WDS_BASE}/getFullTableDownloadCSV/{product_id}/{lang}"
    r = requests.get(url, headers={"Accept": "application/json"}, timeout=60)
    r.raise_for_status()
    j = r.json()
    if j.get("status") != "SUCCESS":
        raise RuntimeError(j)
    return j["object"]

zip_url_levels = get_wds_zip_url(PID_CPI_LEVELS, LANG)
zip_url_weights = get_wds_zip_url(PID_WEIGHTS, LANG)

zip_url_levels, zip_url_weights


('https://www150.statcan.gc.ca/n1/tbl/csv/18100004-eng.zip',
 'https://www150.statcan.gc.ca/n1/tbl/csv/18100007-eng.zip')

In [7]:
def direct_url(product_id: int, lang: str = "en"):
    suffix = "eng" if lang == "en" else "fra"
    return f"https://www150.statcan.gc.ca/n1/tbl/csv/{product_id}-{suffix}.zip"

headers_dl = {"User-Agent": "Mozilla/5.0", "Accept": "*/*"}

try:
    r_levels = requests.get(zip_url_levels, headers=headers_dl, timeout=180)
    r_levels.raise_for_status()
except Exception as e:
    print("WDS falhou no levels, usando direto:", e)
    r_levels = requests.get(direct_url(PID_CPI_LEVELS, LANG), headers=headers_dl, timeout=180)
    r_levels.raise_for_status()

try:
    r_weights = requests.get(zip_url_weights, headers=headers_dl, timeout=180)
    r_weights.raise_for_status()
except Exception as e:
    print("WDS falhou no weights, usando direto:", e)
    r_weights = requests.get(direct_url(PID_WEIGHTS, LANG), headers=headers_dl, timeout=180)
    r_weights.raise_for_status()

len(r_levels.content), len(r_weights.content)


(15050851, 1161014)

In [8]:
z_levels = zipfile.ZipFile(io.BytesIO(r_levels.content))
z_weights = zipfile.ZipFile(io.BytesIO(r_weights.content))

names_levels = z_levels.namelist()
names_weights = z_weights.namelist()

print("LEVELS files:", len(names_levels))
print("WEIGHTS files:", len(names_weights))
print("\nLevels sample:", names_levels[:10])
print("\nWeights sample:", names_weights[:10])


LEVELS files: 2
WEIGHTS files: 2

Levels sample: ['18100004.csv', '18100004_MetaData.csv']

Weights sample: ['18100007.csv', '18100007_MetaData.csv']


In [9]:
csv_levels = [n for n in names_levels if n.lower().endswith(".csv")]
csv_weights = [n for n in names_weights if n.lower().endswith(".csv")]

big_levels = max(csv_levels, key=lambda n: z_levels.getinfo(n).file_size)
big_weights = max(csv_weights, key=lambda n: z_weights.getinfo(n).file_size)

print("Big levels:", big_levels, z_levels.getinfo(big_levels).file_size)
print("Big weights:", big_weights, z_weights.getinfo(big_weights).file_size)

with z_levels.open(big_levels) as f:
    df_cpi = pd.read_csv(f, low_memory=False)

with z_weights.open(big_weights) as f:
    df_w = pd.read_csv(f, low_memory=False)

df_cpi.shape, df_w.shape


Big levels: 18100004.csv 158480155
Big weights: 18100007.csv 16146878


((1134868, 15), (73154, 17))

In [10]:
df_cpi.head(3)


,REF_DATE,GEO,DGUID,Products and product groups,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,1914-01,Canada,2016A000011124,All-items,2002=100,17,units,0,v41690973,2.200,6.0,NaN,NaN,NaN,1
1,1914-01,Canada,2016A000011124,All-items (1992=100),1992=100,7,units,0,v41713403,2.309,7.2,NaN,NaN,t,1
2,1914-01,Canada,2016A000011124,Goods and services,2002=100,17,units,0,v41691221,2.273,6.0,NaN,NaN,t,1


In [11]:
df_w.head(3)


,REF_DATE,GEO,DGUID,Products and product groups,Price period of weight,Geographic distribution of weight,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,1986,Canada,2016A000011124,All-items,Weight at basket link month prices,Distribution to selected geographies,Percent,239,units,0,v91858736,1.1.1.1,100.0,E,NaN,NaN,2
1,1986,Canada,2016A000011124,All-items,Weight at basket link month prices,Distribution to Canada,Percent,239,units,0,v91858737,1.1.1.2,100.0,E,NaN,NaN,2
2,1986,Canada,2016A000011124,All-items,Weight at basket reference period prices,Distribution to selected geographies,Percent,239,units,0,v91858738,1.1.2.1,100.0,E,NaN,NaN,2


In [12]:
df_cpi.columns[:15], df_w.columns[:15]


(Index(['REF_DATE', 'GEO', 'DGUID', 'Products and product groups', 'UOM',
        'UOM_ID', 'SCALAR_FACTOR', 'SCALAR_ID', 'VECTOR', 'COORDINATE', 'VALUE',
        'STATUS', 'SYMBOL', 'TERMINATED', 'DECIMALS'],
       dtype='object'),
 Index(['REF_DATE', 'GEO', 'DGUID', 'Products and product groups',
        'Price period of weight', 'Geographic distribution of weight', 'UOM',
        'UOM_ID', 'SCALAR_FACTOR', 'SCALAR_ID', 'VECTOR', 'COORDINATE', 'VALUE',
        'STATUS', 'SYMBOL'],
       dtype='object'))

In [13]:
[c for c in df_cpi.columns if "REF" in c.upper() or "DATE" in c.upper()][:20]


['REF_DATE']

In [14]:
[c for c in df_cpi.columns if "GEO" in c.upper()][:10]


['GEO']

In [15]:
[c for c in df_cpi.columns if "product" in c.lower()][:20]


['Products and product groups']

In [17]:
[c for c in df_cpi.columns if c.lower() == "value"]

['VALUE']

In [18]:
col_date = "REF_DATE"
col_geo  = "GEO"
col_item = "Products and product groups"
col_val  = "VALUE"

d = df_cpi[df_cpi[col_geo].astype(str).str.strip().eq("Canada")].copy()
d.shape


(186812, 15)

In [19]:
d[col_date].astype(str).head(10).tolist()


['1914-01',
 '1914-01',
 '1914-01',
 '1914-02',
 '1914-02',
 '1914-02',
 '1914-03',
 '1914-03',
 '1914-03',
 '1914-04']

In [20]:
d[col_date].astype(str).tail(10).tolist()


['2025-11',
 '2025-11',
 '2025-11',
 '2025-11',
 '2025-11',
 '2025-11',
 '2025-11',
 '2025-11',
 '2025-11',
 '2025-11']

In [22]:
dt = pd.to_datetime(d[col_date], errors="coerce")
dt.min(), dt.max()


(Timestamp('1914-01-01 00:00:00'), Timestamp('2025-11-01 00:00:00'))

In [23]:
d[col_item].nunique(), d[col_item].value_counts().head(10)

(358,
 Products and product groups
 All-items                                   1343
 Goods and services                          1260
 All-items (1992=100)                        1260
 Rented accommodation                         923
 Lettuce                                      923
 Mortgage interest cost                       923
 Homeowners' replacement cost                 923
 Property taxes and other special charges     923
 Homeowners' home and mortgage insurance      923
 Homeowners' maintenance and repairs          923
 Name: count, dtype: int64)

In [24]:
w = df_w[df_w["GEO"].astype(str).str.strip().eq("Canada")].copy()
w.shape


(8768, 17)

In [25]:
pd.to_datetime(w["REF_DATE"], errors="coerce").min(), pd.to_datetime(w["REF_DATE"], errors="coerce").max()


(Timestamp('1970-01-01 00:00:00.000001986'),
 Timestamp('1970-01-01 00:00:00.000002024'))

In [26]:
last_w_date = pd.to_datetime(w["REF_DATE"], errors="coerce").max()
last_w_date

Timestamp('1970-01-01 00:00:00.000002024')

In [27]:
w_last = w[pd.to_datetime(w["REF_DATE"], errors="coerce") == last_w_date].copy()
w_last.shape

(660, 17)